In [ ]:
# Prerequisites:
# The images are classified as per their labels into CE and LAA folders.
# A matlab script is run to augment the data
# The augmented data is stored in the seperate CE and LAA folders.
# This file runs a script that maps the new and old image names and with 
# respective  labels  

# This file is used for the following two tasks:
#  1. To map the augmented dataset with labels
#  2. Merge the dataset
#  3. Divide the new dataset into testing and training dataset
#  4. This script further tranfers the images into test and training folders

# !pip install opencv-python

In [ ]:
#Importing libraries

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import glob
import random
import os
import cv2
from tqdm import tqdm
import PIL
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import tifffile

In [ ]:
# Change the path
image_path = r'C:\Users\Lenovo\Desktop\ML_coursework\project'
# testimage_path = r'D:\mayo-clinic-strip-ai\test'
# trainimage_path = r'D:\mayo-clinic-strip-ai\train'
# otherimage_path = r'D:\mayo-clinic-strip-ai\other'

In [ ]:
# Change the path
train_df = pd.read_csv(r'D:\mayo-clinic-strip-ai\train.csv')
test_df = pd.read_csv(r'D:\mayo-clinic-strip-ai\test.csv')
other_df = pd.read_csv(r'D:\mayo-clinic-strip-ai\other.csv')
samsubmit_df = pd.read_csv(r'D:\mayo-clinic-strip-ai\sample_submission.csv')


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
other_df.head()

In [ ]:
samsubmit_df.head()

In [ ]:
# Can be commeted 
# ten_img = ['0cc0bc_0', '0ed87f_1', '0ff890_0', '1d5335_0', '03e6b7_0', '032f10_0', '113bb3_0', '0372b0_0',
#            '04439c_0', '09644e_4']
# train_df = train_df[train_df['image_id'].isin(ten_img)]

train_df.head()

In [ ]:
train_path = image_path+'train\\'
train_path

In [ ]:
df= pd.DataFrame()

In [ ]:
# This script looks up a folder and registers all the image names and assigns 
# them a label. You repeat this block for boh CE and LAA augmentation folders
# store path of images

train_images_paths = []
train_images       =[]
w = []
h = []
byt = []
train_path = image_path+'\LAA\\'

for i,img in enumerate(os.listdir(train_path)) :
    image_full_path = train_path+img
    train_images_paths.append(image_full_path)
    train_images.append(img)
    w.append(Image.open(train_path+img).width)
    h.append(Image.open(train_path+img).height)
    byt.append(os.stat(train_path+img).st_size/1000)
#     os.path.getsize(img)
    
    
# add path to data frame
df.loc[:, ('path')] = train_images_paths 
df.loc[:, ('image')] = train_images
df.loc[:, ('width')] = w
df.loc[:, ('height')] = h
df.loc[:, ('KB_size')] = byt
df['label'] = "LAA"
df.head(5)

,path,image,width,height,KB_size,label
0,C:\Users\Lenovo\Desktop\ML_coursework\project\...,00c058_0.jpg,400,400,38.618,LAA
1,C:\Users\Lenovo\Desktop\ML_coursework\project\...,028989_0.jpg,400,400,68.644,LAA
2,C:\Users\Lenovo\Desktop\ML_coursework\project\...,03d1ec_0.jpg,400,400,31.274,LAA
3,C:\Users\Lenovo\Desktop\ML_coursework\project\...,055f6a_0.jpg,400,400,48.259,LAA
4,C:\Users\Lenovo\Desktop\ML_coursework\project\...,062387_0.jpg,400,400,61.154,LAA


In [ ]:
# Write code to Save this CSV
df.to_csv(r'C:\Users\Lenovo\Desktop\ML_coursework\project\LAA\balanced_datasetLAA.csv')

In [ ]:
# After mapping all the new image names with respective labels, combine the csv files and also get all the images into one combined_data folder manually
# Next we will split the augmented dataset into training and testing data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train_new_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\ML_coursework\project\balanced_dataset\combined_data_CSV.csv')
train_new_df.head(10)

,path,image_id,width,height,KB_size,label
0,C:\Users\Lenovo\Desktop\ML_coursework\project\...,00c058_0.jpg,400,400,38.618,LAA
1,C:\Users\Lenovo\Desktop\ML_coursework\project\...,028989_0.jpg,400,400,68.644,LAA
2,C:\Users\Lenovo\Desktop\ML_coursework\project\...,03d1ec_0.jpg,400,400,31.274,LAA
3,C:\Users\Lenovo\Desktop\ML_coursework\project\...,055f6a_0.jpg,400,400,48.259,LAA
4,C:\Users\Lenovo\Desktop\ML_coursework\project\...,062387_0.jpg,400,400,61.154,LAA
5,C:\Users\Lenovo\Desktop\ML_coursework\project\...,08d3d8_0.jpg,400,400,60.592,LAA
6,C:\Users\Lenovo\Desktop\ML_coursework\project\...,098f15_0.jpg,400,400,59.351,LAA
7,C:\Users\Lenovo\Desktop\ML_coursework\project\...,0a3ad1_0.jpg,400,400,42.325,LAA
8,C:\Users\Lenovo\Desktop\ML_coursework\project\...,0a47c9_0.jpg,400,400,68.115,LAA
9,C:\Users\Lenovo\Desktop\ML_coursework\project\...,0bd414_0.jpg,400,400,61.643,LAA


In [ ]:
train_new_df.image_id[9]

'0bd414_0.jpg'

In [ ]:
array = np.array([train_new_df.path ,train_new_df.image_id, train_new_df.width, train_new_df.height, train_new_df.KB_size, train_new_df.label])

np.shape(np.transpose(array))
# np.shape(array)

(2607, 6)

In [ ]:
# Further dividng the training set training and validation

x_train, x_test, y_train, y_test = train_test_split(np.transpose(array),train_new_df.label,test_size = 0.2, train_size = 0.8)

In [ ]:
x_train

array([['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\CE\\b6fffe_0.jpg',
        'b6fffe_0.jpg', 400, 400, 47.359, 'CE'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set114.jpg',
        'set114.jpg', 400, 400, 19.391, 'LAA'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\111.jpg',
        '111.jpg', 400, 400, 28.133, 'LAA'],
       ...,
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\CE\\CEset7126.jpg',
        'CEset7126.jpg', 400, 400, 30.439, 'CE'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\CE\\c4a601_0.jpg',
        'c4a601_0.jpg', 400, 400, 39.767, 'CE'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set1118.jpg',
        'set1118.jpg', 400, 400, 25.336, 'LAA']], dtype=object)

In [ ]:
x_test

array([['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set749.jpg',
        'set749.jpg', 400, 400, 31.156, 'LAA'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\126.jpg',
        '126.jpg', 400, 400, 19.723, 'LAA'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set419.jpg',
        'set419.jpg', 400, 400, 8.236, 'LAA'],
       ...,
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set224.jpg',
        'set224.jpg', 400, 400, 13.306, 'LAA'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set682.jpg',
        'set682.jpg', 400, 400, 17.354, 'LAA'],
       ['C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\LAA\\set436.jpg',
        'set436.jpg', 400, 400, 16.381, 'LAA']], dtype=object)

In [ ]:
y_train

1663     CE
349     LAA
26      LAA
1925     CE
1527     CE
       ... 
1890     CE
2201     CE
2139     CE
1683     CE
325     LAA
Name: label, Length: 2085, dtype: object

In [ ]:
y_test

1312    LAA
43      LAA
823     LAA
1818     CE
2242     CE
       ... 
2323     CE
2568     CE
525     LAA
1197    LAA
842     LAA
Name: label, Length: 522, dtype: object

In [ ]:
np.array(y_test)

array(['LAA', 'LAA', 'LAA', 'CE', 'CE', 'LAA', 'CE', 'CE', 'CE', 'CE',
       'CE', 'CE', 'LAA', 'LAA', 'LAA', 'LAA', 'CE', 'CE', 'LAA', 'CE',
       'CE', 'CE', 'LAA', 'LAA', 'CE', 'LAA', 'LAA', 'LAA', 'LAA', 'CE',
       'LAA', 'LAA', 'CE', 'LAA', 'LAA', 'LAA', 'LAA', 'LAA', 'CE', 'LAA',
       'LAA', 'CE', 'CE', 'LAA', 'CE', 'CE', 'LAA', 'LAA', 'LAA', 'LAA',
       'CE', 'LAA', 'CE', 'LAA', 'CE', 'LAA', 'LAA', 'CE', 'CE', 'LAA',
       'LAA', 'LAA', 'CE', 'LAA', 'LAA', 'CE', 'CE', 'CE', 'CE', 'CE',
       'LAA', 'CE', 'LAA', 'LAA', 'CE', 'LAA', 'CE', 'LAA', 'CE', 'CE',
       'LAA', 'LAA', 'LAA', 'LAA', 'LAA', 'LAA', 'CE', 'LAA', 'CE', 'CE',
       'CE', 'LAA', 'LAA', 'LAA', 'CE', 'LAA', 'CE', 'LAA', 'LAA', 'LAA',
       'LAA', 'CE', 'CE', 'LAA', 'LAA', 'LAA', 'LAA', 'LAA', 'CE', 'LAA',
       'LAA', 'LAA', 'CE', 'LAA', 'LAA', 'LAA', 'CE', 'CE', 'LAA', 'LAA',
       'CE', 'CE', 'CE', 'CE', 'CE', 'CE', 'CE', 'LAA', 'LAA', 'CE', 'CE',
       'CE', 'LAA', 'CE', 'CE', 'LAA', 'LAA', 'CE'

In [ ]:
np.shape(x_train)

(2085, 6)

In [ ]:
np.shape(x_test)

(522, 6)

In [ ]:
x_train[0]

In [ ]:
from numpy import asarray
from numpy import savetxt

In [ ]:
train_from_training_set = x_train
test_from_training_set = x_test

# save to csv file
# savetxt('train_dataset.csv', train_from_training_set, delimiter=',')
# savetxt('test_dataset.csv', test_from_training_set, delimiter=',')

In [ ]:
import csv

In [ ]:
# field names 
# image_id , center_id, patient_id, train_new_df.image_num, train_new_df.path, train_new_df.image, train_new_df.new_path, train_new_df.new_image_name, train_new_df.new_KB_size
fields = ['path','image_id','width','height','KB_size','label'] 
    
# data rows of csv file 
rows = x_train

# name of csv file 
filename = "training_data.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [ ]:
training_records_df = pd.read_csv(r'training_data.csv')

In [ ]:
training_records_df

,path,image_id,width,height,KB_size,label
0,C:\Users\Lenovo\Desktop\ML_coursework\project\...,b6fffe_0.jpg,400,400,47.359,CE
1,C:\Users\Lenovo\Desktop\ML_coursework\project\...,set114.jpg,400,400,19.391,LAA
2,C:\Users\Lenovo\Desktop\ML_coursework\project\...,111.jpg,400,400,28.133,LAA
3,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset3305.jpg,400,400,25.210,CE
4,C:\Users\Lenovo\Desktop\ML_coursework\project\...,5c92f8_0.jpg,400,400,26.142,CE
...,...,...,...,...,...,...
2080,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset3274.jpg,400,400,22.849,CE
2081,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset7182.jpg,400,400,15.123,CE
2082,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset7126.jpg,400,400,30.439,CE
2083,C:\Users\Lenovo\Desktop\ML_coursework\project\...,c4a601_0.jpg,400,400,39.767,CE


In [ ]:
#  Logic flow of the program

# for every entry in the array
# take the first [0th] entry and store it as a string
# Find that file in the folder
# move that into a training folder

In [ ]:
# field names 
# image_id , center_id, patient_id, train_new_df.image_num, train_new_df.path, train_new_df.image, train_new_df.new_path, train_new_df.new_image_name, train_new_df.new_KB_size
fields = ['path','image_id','width','height','KB_size','label'] 
    
# data rows of csv file 
rows = x_test

# name of csv file 
filename = "testing_data.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [ ]:
# field names 
# image_id , center_id, patient_id, train_new_df.image_num, train_new_df.path, train_new_df.image, train_new_df.new_path, train_new_df.new_image_name, train_new_df.new_KB_size
fields = ['label'] 
    
# data rows of csv file 
rows = np.array(y_test)

    
# name of csv file 
filename = "testing_labels.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)

In [ ]:
# field names 
# image_id , center_id, patient_id, train_new_df.image_num, train_new_df.path, train_new_df.image, train_new_df.new_path, train_new_df.new_image_name, train_new_df.new_KB_size
fields = ['label'] 
    
# data rows of csv file 
rows = np.array(y_train)

# name of csv file 
filename = "training_labels.csv"
    
# writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(rows)
    
#     Data resulted into each label getting split into different columns. It is manuall corrected on excel

In [ ]:
#  Assign locations to store the test and train images
new_train_path = r'C:\Users\Lenovo\Desktop\ML_coursework\project\balanced_dataset\train';
new_test_path = r'C:\Users\Lenovo\Desktop\ML_coursework\project\balanced_dataset\test';

train_train_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\ML_coursework\project\balanced_dataset\balanced_training_data.csv')


In [ ]:
train_train_df

,path,image_id,width,height,KB_size,label
0,C:\Users\Lenovo\Desktop\ML_coursework\project\...,b6fffe_0.jpg,400,400,47.359,CE
1,C:\Users\Lenovo\Desktop\ML_coursework\project\...,set114.jpg,400,400,19.391,LAA
2,C:\Users\Lenovo\Desktop\ML_coursework\project\...,111.jpg,400,400,28.133,LAA
3,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset3305.jpg,400,400,25.210,CE
4,C:\Users\Lenovo\Desktop\ML_coursework\project\...,5c92f8_0.jpg,400,400,26.142,CE
...,...,...,...,...,...,...
2080,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset3274.jpg,400,400,22.849,CE
2081,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset7182.jpg,400,400,15.123,CE
2082,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset7126.jpg,400,400,30.439,CE
2083,C:\Users\Lenovo\Desktop\ML_coursework\project\...,c4a601_0.jpg,400,400,39.767,CE


In [ ]:
type(train_train_df.path)

pandas.core.series.Series

In [ ]:
train_train_df.path[0]

'C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\CE\\b6fffe_0.jpg'

In [ ]:
import shutil
#  This allows us to transfer the images from the common folder (in this case the training folder) into a new training and valdiation/test folder

In [ ]:
# Copy the training examples from train folder into another train folder
for entry in train_train_df.path:
#     print(entry)
    image_new_name  = entry.split('\\')[-1].split('.')[0]
    image_new_pathe= 'C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\balanced_dataset\\train'+'\\'+image_new_name+'.jpg'    
#     print(image_new_pathe)
    shutil.copy(entry,image_new_pathe)


In [ ]:


train_test_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\ML_coursework\project\balanced_dataset\balanced_testing_data.csv')


In [ ]:
train_test_df

,path,image_id,width,height,KB_size,label
0,C:\Users\Lenovo\Desktop\ML_coursework\project\...,set749.jpg,400,400,31.156,LAA
1,C:\Users\Lenovo\Desktop\ML_coursework\project\...,126.jpg,400,400,19.723,LAA
2,C:\Users\Lenovo\Desktop\ML_coursework\project\...,set419.jpg,400,400,8.236,LAA
3,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset3209.jpg,400,400,12.101,CE
4,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset7219.jpg,400,400,22.926,CE
...,...,...,...,...,...,...
517,C:\Users\Lenovo\Desktop\ML_coursework\project\...,CEset7292.jpg,400,400,12.504,CE
518,C:\Users\Lenovo\Desktop\ML_coursework\project\...,e7bec8_0.jpg,400,400,25.262,CE
519,C:\Users\Lenovo\Desktop\ML_coursework\project\...,set224.jpg,400,400,13.306,LAA
520,C:\Users\Lenovo\Desktop\ML_coursework\project\...,set682.jpg,400,400,17.354,LAA


In [ ]:
# Copy the testing examples from train folder into test folder
for entry in train_test_df.path:
#     print(entry)
    image_new_name  = entry.split('\\')[-1].split('.')[0]
    image_new_pathe= 'C:\\Users\\Lenovo\\Desktop\\ML_coursework\\project\\balanced_dataset\\test'+'\\'+image_new_name+'.jpg'    
#     print(image_new_pathe)
    shutil.copy(entry,image_new_pathe)


In [ ]:
train